In [11]:
import pandas as pd

df = pd.read_csv('../data/results/testresults.csv')

df[['predictor','depressionSensF1','normalSensF1','depressionOtherF1','normalOtherF1','depressionTotalF1','normalTotalF1','equalAccuracy']]

,predictor,depressionSensF1,normalSensF1,depressionOtherF1,normalOtherF1,depressionTotalF1,normalTotalF1,equalAccuracy
0,w2vec,0.949153,0.940000,0.980392,0.980392,0.958580,0.953642,0.963853
1,ruleBased,0.867925,0.875000,0.844444,0.877193,0.860927,0.875740,1.010217
2,blingfire,0.702381,0.000000,0.575342,0.000000,0.663900,0.000000,1.340323
3,reweightedblingfire,0.702381,0.000000,0.575342,0.000000,0.663900,0.000000,1.340323
4,nltk,0.702381,0.000000,0.575342,0.000000,0.663900,0.000000,1.340323
5,reweightednltk,0.702381,0.000000,0.575342,0.000000,0.663900,0.000000,1.340323
6,spacysm,0.888889,0.871287,0.878049,0.920635,0.886076,0.890244,0.974429
7,reweightedspacysm,0.862069,0.843137,0.820513,0.892308,0.851613,0.862275,0.985933
8,spacylg,0.854701,0.831683,0.878049,0.920635,0.860759,0.865854,0.933828
9,reweightedspacylg,0.852174,0.834951,0.829268,0.888889,0.846154,0.855422,0.975331
